In [ ]:
pip install mysqlclient

In [ ]:
pip install pymysql

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from pydataset import data

In [2]:
from env import host, user, password

url = f'mysql+pymysql://{user}:{password}@{host}/employees'

In [ ]:
query = """SELECT * FROM salaries ORDER BY salary DESC LIMIT 10"""
df = pd.read_sql(query, url)
df

In [ ]:
# 1 Load the mpg dataset. Read the documentation for it, and use the data to answer these questions:
from pydataset import data
mpg = data('mpg') # load the dataset and store it in a variable
# data('mpg', show_doc=True) # view the documentation for the dataset
cars = pd.DataFrame(mpg)

In [ ]:
# On average, which manufacturer has the best miles per gallon?
print(cars.info())
manufacturers = cars.groupby("manufacturer").mean()
cars['average_mileage'] = (cars.cty + cars.hwy) / 2
cars
best_mpg = manufacturers.sort_values(by='average_mileage', ascending = False)
best_mpg

In [ ]:
best_mpg.head(1)

In [ ]:
#1 B.How many different manufacturers are there?
len(cars["manufacturer"].value_counts())


In [ ]:
#1 C. How many different models are there?
len(cars["model"].value_counts())

In [ ]:
#1 D. Do automatic or manual cars have better miles per gallon? Manual trans has better mpg
automatic = cars[cars.trans.str.contains("auto")]
print(automatic.average_mileage.mean())
manual = cars[cars.trans.str.contains("m")]
print(manual.average_mileage.mean())



In [ ]:
#2 Joining and Merging
# Copy the users and roles dataframes from the examples above. What do you think a right join 
# would look like? An outer join? What happens if you drop the foreign keys from the dataframes 
# and try to merge them?
users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles
# Right Join shows all the columns in the right data frame. If data is missing on the left DF
#NaN will populate
users_w_roles = pd.merge(users, roles, left_on='role_id', right_on='id', how='right')
users_w_roles

In [ ]:
#Outer Join
users_w_roles = pd.merge(users, roles, left_on='role_id', right_on='id', how='outer')
users_w_roles

In [ ]:
# What happens if you drop the foreign keys from the dataframes and try to merge them?
users = users.drop(columns=['role_id'])
users
pd.merge(users, roles, left_on = 'role_id', right_on='id') # cannot merge as we dropped role_id and 
                                                            # cannot merge on the role_id

In [4]:
#3 Getting data from SQL databases
# A.Create a function named get_db_url. It should accept a username, hostname, password, and 
# database name and return a url formatted like in the examples in this lesson.

def get_db_url(db):   ##Why does db need to be in the function? It will not work otherwise
    from env import host, user, password
    url = (f'mysql+pymysql://{user}:{password}@{host}/{db}')
    return url

In [ ]:
# Showing all databases
query = '''show databases'''
pd.read_sql(query, url)

In [9]:
# 3B. Use your function to obtain a connection to the employees database.
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', url)
# or
pd.read_sql('SELECT * FROM employees LIMIT 5 OFFSET 50', get_db_url('employees'))

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10051,1953-07-28,Hidefumi,Caine,M,1992-10-15
1,10052,1961-02-26,Heping,Nitsch,M,1988-05-21
2,10053,1954-09-13,Sanjiv,Zschoche,F,1986-02-04
3,10054,1957-04-04,Mayumi,Schueller,M,1995-03-13
4,10055,1956-06-06,Georgy,Dredge,M,1992-04-27


In [ ]:
#3C. Intentionally make an error in your SQL query. What does the error message look like?
def bad_url(un,pas,hos,db):
    url = (f'mysql+pymysql://{user}:{password}@{host}/{db}')
    return url
pd.read_sql('SELECT * FROsM employees LIMIT 5 OFFSET 50', bad_url(user, host, password, 'employees'))

# w/ bad FROsM statement 
#ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'FROsM employees LIMIT 5 OFFSET 50' at line 1")
#[SQL: SELECT * FROsM employees LIMIT 5 OFFSET 50]
#(Background on this error at: http://sqlalche.me/e/f405)


In [10]:
#3D. Read the employees and titles tables into two separate dataframes
employees = pd.read_sql('SELECT * FROM employees', get_db_url('employees'))
employees
titles = pd.read_sql('SELECT * FROM titles', get_db_url('employees'))
titles

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


In [11]:
#3E. Visualize the number of employees with each title.
sql = """SHOW TABLES"""
pd.read_sql(sql, url)
print(titles.head(5)) # titles has emp_no, title, from_date, & to_date
print(employees.head(5)) #employees has emp_no, birth_date, first_name, last_name, gender and hire_date

   emp_no            title   from_date     to_date
0   10001  Senior Engineer  1986-06-26  9999-01-01
1   10002            Staff  1996-08-03  9999-01-01
2   10003  Senior Engineer  1995-12-03  9999-01-01
3   10004         Engineer  1986-12-01  1995-12-01
4   10004  Senior Engineer  1995-12-01  9999-01-01
   emp_no  birth_date first_name last_name gender   hire_date
0   10001  1953-09-02     Georgi   Facello      M  1986-06-26
1   10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
2   10003  1959-12-03      Parto   Bamford      M  1986-08-28
3   10004  1954-05-01  Chirstian   Koblick      M  1986-12-01
4   10005  1955-01-21    Kyoichi  Maliniak      M  1989-09-12


In [13]:
#going to join tables to get a count of employee titles
current_titles = pd.read_sql('SELECT * FROM titles WHERE to_date > CURDATE()', get_db_url('employees'))
current_titles # getting current titles WHERE to_date > CURDATE()
#Joining current titles with employees
emp_w_current_titles = pd.merge(current_titles, employees, left_on='emp_no', right_on='emp_no', how='right')
#value counts shows us the counts of the distinct title names
#getting value count of title on the merged tables
emp_w_current_titles.title.value_counts()

Senior Engineer       85939
Senior Staff          82024
Engineer              30983
Staff                 25526
Technique Leader      12055
Assistant Engineer     3588
Manager                   9
Name: title, dtype: int64

In [ ]:
#Alt way to get current employees titles
cur_titles = titles[titles['to_date'] >= pd.to_datetime('today')] # current employees titles
cur_titles
#grouping by title in current titles. Doing a count on the emp_no
count_titles = cur_titles.groupby('title').emp_no.agg(['count']) 
count_titles

In [14]:
#3F. Join the employees and titles dataframes together.
# merging employees and titles on emp_no, showing all columns of data in each
print(titles.head)
print(employees.head)
emp_w_titles = pd.merge(employees, titles, left_on = 'emp_no', right_on = 'emp_no', how = "inner")
emp_w_titles


<bound method NDFrame.head of         emp_no            title   from_date     to_date
0        10001  Senior Engineer  1986-06-26  9999-01-01
1        10002            Staff  1996-08-03  9999-01-01
2        10003  Senior Engineer  1995-12-03  9999-01-01
3        10004         Engineer  1986-12-01  1995-12-01
4        10004  Senior Engineer  1995-12-01  9999-01-01
...        ...              ...         ...         ...
443303  499997         Engineer  1987-08-30  1992-08-29
443304  499997  Senior Engineer  1992-08-29  9999-01-01
443305  499998     Senior Staff  1998-12-27  9999-01-01
443306  499998            Staff  1993-12-27  1998-12-27
443307  499999         Engineer  1997-11-30  9999-01-01

[443308 rows x 4 columns]>
<bound method NDFrame.head of         emp_no  birth_date first_name last_name gender   hire_date
0        10001  1953-09-02     Georgi   Facello      M  1986-06-26
1        10002  1964-06-02    Bezalel    Simmel      F  1985-11-21
2        10003  1959-12-03      Parto  

,emp_no,birth_date,first_name,last_name,gender,hire_date,title,from_date,to_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26,Senior Engineer,1986-06-26,9999-01-01
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21,Staff,1996-08-03,9999-01-01
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28,Senior Engineer,1995-12-03,9999-01-01
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Engineer,1986-12-01,1995-12-01
4,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...,...,...,...,...,...
443303,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Engineer,1987-08-30,1992-08-29
443304,499997,1961-08-03,Berhard,Lenart,M,1986-04-21,Senior Engineer,1992-08-29,9999-01-01
443305,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Senior Staff,1998-12-27,9999-01-01
443306,499998,1956-09-05,Patricia,Breugel,M,1993-10-13,Staff,1993-12-27,1998-12-27


In [ ]:
#3G Visualize how frequently employees change titles.
emp_w_current_titles.plot.barh(width = .7)
plt.title("Employees Current Titles")
plt.xlabel("Number of Employees")
#inverting y axis for horizontal bar chart
plt.gca().invert_yaxis()

plt.show()

In [ ]:
#3H. For each title, find the hire date of the employee that was hired most recently with that title..max
emp_w_titles.groupby('title')['hire_date'].max()

In [ ]:
#3H alt
titles.groupby('title').from_date.max()

In [ ]:
#3I. Write the code necessary to create a cross tabulation of the number of titles by department. 
# (Hint: this will involve a combination of SQL and python/pandas code)
#Making dept emp DF to get dept names to tie to titles
dept_emp = pd.read_sql('SELECT * FROM dept_emp', get_db_url('employees'))
dept_emp
# Making depts DF to get departments from dept number
depts = pd.read_sql('SELECT * FROM dept_emp', get_db_url('employees'))
depts
# Current titles DF. Created up top
cur_titles = titles[titles['to_date'] >= pd.to_datetime('today')]
# Current departments DF
cur_dept = dept_emp[dept_emp['to_date'] >= pd.to_datetime('today')]
# Joining depts and cur_dept to see current departments along with department numbers
depts_n_cur_depts = pd.merge(depts,cur_dept, left_on = 'dept_no', right_on = "dept_no", how = 'inner')
# Joining depts_n_cur_depts with cur_titles
depts_w_titles = pd.merge(depts_n_cur_depts, cur_titles, left_on = 'emp_no', right_on = 'emp_no', how = 'inner')
# Crosstab of dept_name and titles
depts_w_titles_crossstab = pd.crosstab(dept_w_titles.dept_name,depts_w_titles.titles)
depts_w_titles_crosstab



In [5]:
# 4 Use your get_db_url function to help you explore the data from the chipotle database. 
# Use the data to answer the following questions:
orders = pd.read_sql('''SELECT * FROM orders''',get_db_url('chipotle'))
orders.head()

#A What is the total price for each order?
#Making item price to show as a float
orders['item_price'] = orders.item_price.str.replace('$','').astype(float)
# sum up the total of quantity  * item price
orders["total"] = orders.quantity * orders.item_price
orders

,id,order_id,quantity,item_name,choice_description,item_price,total
0,1,1,1,Chips and Fresh Tomato Salsa,nan,2.39,2.39
1,2,1,1,Izze,[Clementine],3.39,3.39
2,3,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,2.39,2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",16.98,33.96
...,...,...,...,...,...,...,...
4617,4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,11.75
4618,4619,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese, Lettuce, Guacamole]]",11.75,11.75
4619,4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto Beans, Guacamole, Lettuce]]",11.25,11.25
4620,4621,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettuce]]",8.75,8.75


In [6]:
#B What are the most popular 3 items?
orders.item_name.value_counts().head(3)



Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

In [8]:
#C Which item has produced the most revenue?
totals = orders.groupby('item_name').total.sum()
totals = totals.sort_values(ascending=False).head(1)
round(totals.max(),2)
totals

item_name
Chicken Bowl    8044.63
Name: total, dtype: float64